LSTM RNN Demo for predicting pronouns 

In [2]:
from __future__ import division, print_function, absolute_import
import tflearn
from tflearn.data_utils import to_categorical, pad_sequences
import numpy as np
import pandas as pd


In [5]:
df = pd.read_csv("/Users/richfisher/projects/cortazar_data/test_data_final_state/adventure_genre_with_ant_labels.csv")
df.shape

(60811, 3)

In [31]:
df[8000:8010] 

,Unnamed: 0,text,val
8000,8000,[A] wasn't sure what [???] wanted [A] to recal...,[B]
8001,8001,[A] wasn't sure what [B] wanted [???] to recal...,[A]
8002,8002,[A] wasn't sure what [B] wanted [A] to recall ...,[B]
8003,8003,"Though the clouds darken the sun... "" [???] wo...",[A]
8004,8004,"Though the clouds darken the sun... "" [A] word...",[B]
8005,8005,"Though the clouds darken the sun... "" [A] word...",[B]
8006,8006,"Though the clouds darken the sun... "" [A] word...",[B]
8007,8007,"Though the clouds darken the sun... "" [A] word...",[A]
8008,8008,"Though the clouds darken the sun... "" [A] word...",[B]
8009,8009,[A]-ant opened [???] eyes and found [B]-ant st...,[A]


In [32]:
X = df.text.tolist()
y = df.val.tolist()

In [36]:
def run_le_lstm(xset, yset, split_n):
    dictionary = {}
    i = 1
    for sent in xset:
        for word in sent.split(" "):
            if word not in dictionary:
                dictionary[word] = i
                i += 1
    print("dicionatry len", len(dictionary))

    # #https://github.com/tflearn/tflearn/blob/master/examples/nlp/lstm.py
    parsed_xset = []
    for sent in xset:
        sent_parsed = []
        for word in sent.split(" "):
            sent_parsed.append(dictionary[word])
        parsed_xset.append(sent_parsed)


    x_train = parsed_xset[:split_n] #50k #adapt to variable data sizes
    x_test = parsed_xset[split_n:] #10k

    trainX = pad_sequences(x_train, maxlen=43, value=0.)
    testX = pad_sequences(x_test, maxlen=43, value=0.)

    binary_yset = [1 if x == '[B]' else 0 for x in yset]
    y_train = binary_yset[:split_n]
    y_test = binary_yset[split_n:]

    trainY = to_categorical(y_train, nb_classes=2)
    testY = to_categorical(y_test, nb_classes=2)

    net = tflearn.input_data([None, 43])
    net = tflearn.embedding(net, input_dim=len(dictionary)+1, output_dim=128)
    net = tflearn.lstm(net, 128, dropout=0.8)
    net = tflearn.fully_connected(net, 2, activation='softmax')
    net = tflearn.regression(net, optimizer='adam', learning_rate=0.001, \
                             loss='categorical_crossentropy')
    model = tflearn.DNN(net, tensorboard_verbose=0)
    model.fit(trainX, trainY, validation_set=(testX, testY), show_metric=True, \
              batch_size=60)
    
    

In [37]:
# FYI in console this will print out results from ALL epochs AND will show VALIDATION accuracy (not just training accuracy as depicted here). 

run_le_lstm(X, y, 50000)


Training Step: 3682  | total loss: 0.51236
| Adam | epoch: 005 | loss: 0.51236 - acc: 0.7389 -- iter: 20760/50000


KeyboardInterrupt: 